In [1]:
#Random forest
#https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/2_BasicModels/random_forest.py#L1
import warnings
warnings.filterwarnings('ignore')

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#This line is neccessarry to avoid the error of reusing a variable in tensor flow.
tf.reset_default_graph()

# Parameters
num_steps = 500 # Total steps to train
batch_size = 1024 # The number of samples per batch
num_classes = 10 # The 10 digits
num_features = 784 # Each image is 28x28 pixels
num_trees = 10
max_nodes = 1000

# Input and Target data
X = tf.placeholder(tf.float32, shape=[None, num_features])
# For random forest, labels must be integers (the class id)
Y = tf.placeholder(tf.int32, shape=[None])

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(),
    resources.initialize_resources(resources.shared_resources()))

# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars)

# Training
for i in range(1, num_steps + 1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

# Test Model
test_x, test_y = mnist.test.images, mnist.test.labels
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_outputs': 1, 'bagging_fraction': 1.0, 'split_type': 0, 'num_classes': 10, 'split_name': 'less_or_equal', 'collate_examples': False, 'early_finish_check_every_samples': 0, 'valid_leaf_threshold': 1, 'num_features': 784, 'pruning_type': 0, 'dominate_fraction': 0.99, 'finish_type': 0, 'max_fertile_nodes': 0, 'inference_tree_paths': False, 'split_pruning_name': 'none', 'leaf_model_type': 0, 'bagged_features': None, 'num_trees': 10, 'feature_bagging_fraction': 1.0, 'use_running_stats_method': False, 'base_random_seed': 0, 'stats_model_type': 0, 'num_splits_to_consider': 28, 'param_file': None, 'checkpoint_stats': False, 'dominate_method': 'bootstrap', 'max_nodes': 1000, 'prune_every_samples': 0, 'num_output_columns': 11, 'model_name': 'all_dense', 'initialize_average_splits': False, 'split_finish_name': 'basic', 'bagged_num_features': 784, 'split_after_samples': 250, 'regression': False}
Step 1, Loss: -1.000000, Acc: 0

In [3]:
image1_= X[0]
print (image1_)
import PIL
import numpy as np
import matplotlib.pyplot as plt
plt.imshow(image1_.reshape((28,28)), cmap='Greys')

Tensor("strided_slice:0", shape=(784,), dtype=float32)


AttributeError: 'Tensor' object has no attribute 'reshape'